In [ ]:
!pip install tensorflow pandas numpy scikit-learn


In [ ]:
import pandas as pd
import numpy as np

# Download MovieLens 100k
!wget -nc https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

# Load data
df = pd.read_csv("ml-100k/u.data", sep="\t", names=["userId", "movieId", "rating", "timestamp"])
df = df.drop("timestamp", axis=1)

# Reduce to 10k rows
df = df.sample(10000, random_state=42)

# Encode users and movies
user_ids = df["userId"].unique()
movie_ids = df["movieId"].unique()

user_map = {u: i for i, u in enumerate(user_ids)}
movie_map = {m: i for i, m in enumerate(movie_ids)}

df["user"] = df["userId"].map(user_map)
df["movie"] = df["movieId"].map(movie_map)

df.head()


--2025-10-27 14:04:07--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.51MB/s    in 1.3s    

2025-10-27 14:04:09 (3.51 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.ba

,userId,movieId,rating,user,movie
75721,877,381,4,0,0
80184,815,602,3,1,1
19864,94,431,4,2,2
76699,416,875,2,3,3
92991,500,182,2,4,4


In [ ]:
from sklearn.model_selection import train_test_split

X = df[["user", "movie"]]
y = df["rating"].astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

num_users = len(user_ids)
num_movies = len(movie_ids)

user_input = layers.Input(shape=(1,))
movie_input = layers.Input(shape=(1,))

user_embed = layers.Embedding(num_users, 32)(user_input)
movie_embed = layers.Embedding(num_movies, 32)(movie_input)

user_vec = layers.Flatten()(user_embed)
movie_vec = layers.Flatten()(movie_embed)

concat = layers.Concatenate()([user_vec, movie_vec])
dense = layers.Dense(64, activation="relu")(concat)
dense = layers.Dense(32, activation="relu")(dense)
output = layers.Dense(1)(dense)

model = keras.Model([user_input, movie_input], output)
model.compile(optimizer="adam", loss="mse", metrics=["mse"])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 32)     │     28,960 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 32)     │     39,776 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 32)        │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 32)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         33 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 75,009 (293.00 KB)

 Trainable params: 75,009 (293.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    [X_train["user"], X_train["movie"]],
    y_train,
    epochs=5,
    batch_size=64,
    validation_data=([X_test["user"], X_test["movie"]], y_test),
    verbose=1
)


Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 9.2258 - mse: 9.2258 - val_loss: 1.0939 - val_mse: 1.0939
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.9233 - mse: 0.9233 - val_loss: 1.0081 - val_mse: 1.0081
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.7775 - mse: 0.7775 - val_loss: 1.0045 - val_mse: 1.0045
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.7305 - mse: 0.7305 - val_loss: 1.0135 - val_mse: 1.0135
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.7064 - mse: 0.7064 - val_loss: 1.0151 - val_mse: 1.0151


In [ ]:
test_user = df["user"].sample(1).values[0]
all_movies = np.array(list(movie_map.values()))
user_array = np.full_like(all_movies, test_user)

preds = model.predict([user_array, all_movies]).flatten()
top5_ids = preds.argsort()[-5:][::-1]

print("Recommended Movie IDs:", all_movies[top5_ids])


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Recommended Movie IDs: [  22  277  533   37 1127]


In [ ]:
model.save("Movie_Recommendation_NCF.keras")
print("Model saved successfully ✅")


Model saved successfully ✅
